In [3]:
import torch
import os
import numpy as np
from math import sqrt
import csv

exp_dir = "../experiments/experiment_v1/"
max_hv_dir = "./max_hvs/"
output_path = "./final_outputs.csv"

experiment_dict = {
    "1d_toy": {"title": "Toy Problem", "max_hv_name": "1d_toy"},
    "1d_toy_q2": {"title": "Toy Problem, q=2", "max_hv_name": "1d_toy"},
    "1d_toy_q4": {"title": "Toy Problem, q=4", "max_hv_name": "1d_toy"},
    "1d_toy_q8": {"title": "Toy Problem, q=8", "max_hv_name": "1d_toy"},
    "bc_v2": {"title": "Constrained Branin Currin", "max_hv_name": "bc_v2"},
    "bc_q2": {"title": "Constrained Branin Currin, q=2", "max_hv_name": "bc_v2"},
    "bc_q4": {"title": "Constrained Branin Currin, q=4", "max_hv_name": "bc_v2"},
    "bc_q8": {"title": "Constrained Branin Currin, q=8", "max_hv_name": "bc_v2"},
    "bc_8": {"title": "Constrained Branin Currin, n_xi=8", "max_hv_name": "bc_v2"},
    "bc_16": {"title": "Constrained Branin Currin, n_xi=16", "max_hv_name": "bc_v2"},
    "bc_64": {"title": "Constrained Branin Currin, n_xi=64", "max_hv_name": "bc_v2"},
    "bc_96": {"title": "Constrained Branin Currin, n_xi=96", "max_hv_name": "bc_v2"},
    "bc_128": {"title": "Constrained Branin Currin, n_xi=128", "max_hv_name": "bc_v2"},
    "bc_heteroskedastic_v2": {"title": "Constrained Branin Currin, Heteroscedastic", "max_hv_name": "bc_heteroskedastic_v2"},
    "disc_brake": {"title": "Disc Brake", "max_hv_name": "disc_brake"},
    "disc_brake_8": {"title": "Disc Brake, n_xi=8", "max_hv_name": "disc_brake"},
    "disc_brake_16": {"title": "Disc Brake, n_xi=16", "max_hv_name": "disc_brake"},
    "disc_brake_64": {"title": "Disc Brake, n_xi=64", "max_hv_name": "disc_brake"},
    "disc_brake_96": {"title": "Disc Brake, n_xi=64", "max_hv_name": "disc_brake"},
    "disc_brake_128": {"title": "Disc Brake, n_xi=128", "max_hv_name": "disc_brake"},
    "penicillin_v2": {"title": "Penicillin"},
    "gmm_demo": {"title": "GMM, Std=0.05", "max_hv_name": "gmm2"},
    "gmm_noise2x": {"title": "GMM, Std=0.10"},
    "gmm_noise4x": {"title": "GMM, Std=0.20"},
    "gmm_hetero": {"title": "GMM, Std=0.2X"},
    "gmm_corr": {"title": "GMM, Correlated"},
    "gmm_mul": {"title": "GMM, Multiplicative"},
    "gmm3": {"title": "GMM, M=3, alpha=0.9"},
    "gmm3_08": {"title": "GMM, M=3, alpha=0.8"},
    "gmm3_07": {"title": "GMM, M=3, alpha=0.7"},
    "gmm_4obj": {"title": "GMM, M=4, alpha=0.9"},
    "gmm_4obj_08": {"title": "GMM, M=4, alpha=0.8"},
}

algorithms = [
    ("sobol", "Sobol"),
    ("nparego", "qNParEGO"),
    ("nehvi", "qNEHVI"),
    ("nehvi_rff", "qNEHVI-RFF"),
    ("expectation_nparego", "Exp-qNParEGO"),
    ("expectation_nehvi_rff", "Exp-NEHVI-RFF"),
    ("ch-var-ucb", "MARS-UCB"),
    ("ref_ch-var-ts", "MARS-TS"),
    ("ref_ch-var-nei", "MARS-NEI"),
    ("ref_mvar_nehvi", "MVaR-NEHVI"),
    ("ref_mvar_nehvi_rff", "MVaR-NEHVI-RFF"),
    ("cas", "Constraint Active Search"),
]

N_REPS = 20
plot_regret = True
min_iter = 100

def save_all():
    f = open(output_path, 'w')
    writer = csv.writer(f)
    writer.writerow(
        [
            "experiment",
            "title",
            "label",
            "alg_name",
            "final_hv_regret",
            "hv_regret_2_sem",
            "per_evaluation_time",
            "time_2_sem",
            "num_replications",
        ]
    )

    for experiment, kwargs in experiment_dict.items():
        q = int(experiment[-1]) if experiment[-2] == "q" else 1
        effective_min_iter = min_iter / q
        if plot_regret:
            max_hv_name = kwargs.get("max_hv_name", experiment)
            max_hv = torch.load(
                os.path.join(max_hv_dir, f"{max_hv_name}.pt")
            )["max_hv"]
        for label, alg_name in algorithms:
            print(f"Processing experiment {experiment}, label {label}.")
            agg_results = {}
            for seed in range(N_REPS):
                path = os.path.join(exp_dir, experiment, label, f"{str(seed).zfill(4)}_{label}.pt")
                try:
                    res = torch.load(path, map_location="cpu")
                except (FileNotFoundError, EOFError):
                    continue
                if label not in agg_results:
                    agg_results[label] = {k: (v.unsqueeze(0) if torch.is_tensor(v) else v) for k,v in res.items()}
                else:
                    label_res = agg_results[label]
                    for k,v in res.items():
                        if torch.is_tensor(v):
                            if v.shape[0] < effective_min_iter:
                                break
                            if v.shape[0] != label_res[k].shape[1]:
                                max_iter = min(label_res[k].shape[1], v.shape[0])
                                label_res[k] = label_res[k][:, :max_iter]
                                v = v[:max_iter]
                            label_res[k] = torch.cat([label_res[k], v.unsqueeze(0)], dim=0)
            try:
                res = agg_results[label]
            except KeyError:
                continue
            hvs = res["all_mvar_hvs"]
            hv_mean = hvs.mean(dim=0)[-1]
            hv_sem = (hvs.std(dim=0)/sqrt(hvs.shape[0]))[-1] * 2
            if plot_regret:
                hv_mean = max_hv - hv_mean
            # Dividing by 100 / 10 to get per-iteration times
            mean_time = res['wall_time'].mean(dim=0)[-1] / 100
            sem_time = res['wall_time'].std(dim=0)[-1] / sqrt(hvs.shape[0]) * 2 / 10
            writer.writerow(
                [
                    experiment,
                    kwargs.get("title"),
                    label,
                    alg_name,
                    hv_mean.item(),
                    hv_sem.item(),
                    mean_time.item(),
                    sem_time.item(),
                    hvs.shape[0],
                ]
            )
    f.close()
    print("Done!")

In [4]:
save_all()

Processing experiment 1d_toy, label sobol.
Processing experiment 1d_toy, label nparego.
Processing experiment 1d_toy, label nehvi.
Processing experiment 1d_toy, label nehvi_rff.
Processing experiment 1d_toy, label expectation_nparego.
Processing experiment 1d_toy, label expectation_nehvi_rff.
Processing experiment 1d_toy, label ch-var-ucb.
Processing experiment 1d_toy, label ref_ch-var-ts.
Processing experiment 1d_toy, label ref_ch-var-nei.
Processing experiment 1d_toy, label ref_mvar_nehvi.
Processing experiment 1d_toy, label ref_mvar_nehvi_rff.
Processing experiment 1d_toy, label cas.
Processing experiment 1d_toy_q2, label sobol.
Processing experiment 1d_toy_q2, label nparego.
Processing experiment 1d_toy_q2, label nehvi.
Processing experiment 1d_toy_q2, label nehvi_rff.
Processing experiment 1d_toy_q2, label expectation_nparego.
Processing experiment 1d_toy_q2, label expectation_nehvi_rff.
Processing experiment 1d_toy_q2, label ch-var-ucb.
Processing experiment 1d_toy_q2, label ref_